In [ ]:
import os
from data_gatherer.data_gatherer import DataGatherer
import pandas as pd
from data_gatherer.llm.response_schema import *
from scripts.experiment_utils import *

In [ ]:
input_file = "scripts/exp_input/REV.txt"

In [ ]:
model_name = "gpt-4o-mini"  # "gemini-2.0-flash" or "gpt-4o-mini" or (bad) "qwen:4b" or "gemma3:4b"
prompt = "GPT_FewShot"  # "GPT_from_full_input_Examples" or "GPT_FewShot" or "Ollama_FewShot"
FDR = False
semantic_retrieval = True
section_filter= None


In [ ]:
# read list from a text file
with open(input_file, 'r') as f:
    pmcids = f.read().splitlines()

print("Number of PMCIDs:", len(pmcids))

In [ ]:
dg = DataGatherer(
    llm_name=model_name,
    log_level='INFO', 
    process_entire_document=FDR, 
    driver_path='../Firefox/geckodriver', 
    save_to_cache=True, 
    load_from_cache=True
) #, save_dynamic_prompts=True

In [ ]:
# These are the problematic urls for information extraction for GEMINI
#urls = ["https://pmc.ncbi.nlm.nih.gov/articles/PMC9314356", "https://pmc.ncbi.nlm.nih.gov/articles/PMC4318527/", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11929800/", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11032436/", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10802452/", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8565335/","https://pmc.ncbi.nlm.nih.gov/articles/PMC7778917/"]
# v1 = "https://pmc.ncbi.nlm.nih.gov/articles/PMC7029360/"
# v2 = "https://pmc.ncbi.nlm.nih.gov/articles/PMC8006362"
#urls = ["https://pmc.ncbi.nlm.nih.gov/articles/PMC9710693/"]

# These are the problematic urls for information extraction for GPT - (2025 Aug 29) GPT gets stuck for some reason after 50 calls or so...
#urls = ["https://pmc.ncbi.nlm.nih.gov/articles/PMC11240079", "https://pmc.ncbi.nlm.nih.gov/articles/PMC5752539","https://pmc.ncbi.nlm.nih.gov/articles/PMC7032692", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10547713", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8102856", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8055881", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8226229", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10769298", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11659981", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8082263", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8565335", "https://pmc.ncbi.nlm.nih.gov/articles/PMC9094742", "https://pmc.ncbi.nlm.nih.gov/articles/PMC7029360", "https://pmc.ncbi.nlm.nih.gov/articles/PMC6289083", "https://pmc.ncbi.nlm.nih.gov/articles/PMC7658217", "https://pmc.ncbi.nlm.nih.gov/articles/PMC9780309","https://pmc.ncbi.nlm.nih.gov/articles/PMC3788619", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8859891", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10680627", "https://pmc.ncbi.nlm.nih.gov/articles/PMC6323985", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8131595", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11420198", "https://pmc.ncbi.nlm.nih.gov/articles/PMC9915613", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10238095", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10802452" , "https://pmc.ncbi.nlm.nih.gov/articles/PMC11032436", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10329279", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10836119",  "https://pmc.ncbi.nlm.nih.gov/articles/PMC9280291", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11661334", "https://pmc.ncbi.nlm.nih.gov/articles/PMC5161470", "https://pmc.ncbi.nlm.nih.gov/articles/PMC4339277"]

#out = dg.process_articles(urls, semantic_retrieval=semantic_retrieval, section_filter=section_filter, prompt_name=prompt)

#out
#out['https://pmc.ncbi.nlm.nih.gov/articles/PMC9710693']

In [ ]:
combined_df = dg.run(
    input_file=pmcids, 
    semantic_retrieval=semantic_retrieval, 
    section_filter=section_filter, 
    prompt_name=prompt,
    full_output_file='scripts/output/result_REV_RTR_gpt.csv',
    top_k=3
    )

In [ ]:
combined_df = pd.read_csv("scripts/output/result_REV_RTR_gpt.csv")
combined_df.head()

In [ ]:
#ground_truth = pd.read_parquet("scripts/exp_input/dataset_citation_records_Table.parquet")
ground_truth = pd.read_parquet("scripts/output/gold/dataset_citation_records_Table.parquet")
#ground_truth = pd.read_excel("scripts/output/gold/GT_DataLinks_PMC_and_Fenyo.xlsx")
#ground_truth = df

In [ ]:
ground_truth.head()
# rename colum synid to 'identifier'
#ground_truth = ground_truth.rename(columns={'synid': 'identifier'})

In [ ]:
results = evaluate_performance(combined_df, ground_truth, dg, "scripts/output/false_positives.txt", "scripts/output/false_negatives.txt")

In [ ]:
results

In [ ]:
dg.input_tokens_total